In [6]:
from needleman_wunsh import make_alignment


def calculate_score(seq1: str, seq2: str, match: int, mismatch: int, gap: int) -> int:
    """Calculates score of alignment. Helper function used for results validation. """

    def s(a, b):
        """Score of pair"""
        if a == '-' or b == '-':
            return gap
        elif a == b:
            return match
        else:
            return mismatch

    score = 0
    for i in range(len(seq1)):
        score += s(seq1[i], seq2[i])
    return score


# Sequences to be aligned
seq1 = 'GGATCGAAGCIBBBWYSIOASUHDJSGCGT'
seq2 = 'GGAAGCIBBBBWYSIOASFSHI'
# Alignment parameters
match = 1
mismatch = -2
gap = -1

# Make an alignment
aligned_seq1, aligned_seq2, score = make_alignment(seq1, seq2, match=match, mismatch=mismatch, gap=gap)

print(f'seq1: {aligned_seq1}')
print(f'seq2: {aligned_seq2}')
print()
print(f'Alignment score: {score}')

# Test correctness
dealigned_seq1 = aligned_seq1.replace('-', '')
dealigned_seq2 = aligned_seq2.replace('-', '')

assert len(aligned_seq1) == len(aligned_seq2), f'aligned sequences must have the same lenghts'
assert len(seq1) == len(dealigned_seq1), f'Length of seq1 does not match length of aligned_seq1: {len(seq1)} !=  {len(dealigned_seq1)}'
assert len(seq2) == len(dealigned_seq2), f'Length of seq1 does not match length of aligned_seq1: {len(seq2)} !=  {len(dealigned_seq2)}'
assert dealigned_seq1 == seq1, f'Output seq1 is different than input seq1'
assert dealigned_seq2 == seq2, f'Output seq2 is different than input seq2'
for i in range(len(aligned_seq1)):
    assert not (aligned_seq1[i] == '-' and aligned_seq2[i] == '-'), f'alignment can not have dash in both sequences at the same position: {i}'
actual_score = calculate_score(aligned_seq1, aligned_seq2, match, mismatch, gap)
assert actual_score == score, f'Reported score does not match actual score: {score} != {actual_score}'

print()
print('Looks fine :)')

seq1: GGATCGAAGCI-BBBWYSIOASUHDJSGCGT
seq2: -G---GAAGCIBBBBWYSIOAS---FS--HI

Alignment score: 2.0

Looks fine :)


In [11]:
import numpy as np
def make_alignment(s1, s2, match, mismatch, gap):
    s = np.zeros((len(s1)+1,len(s2)+1)) # score
    d = np.zeros((len(s1)+1,len(s2)+1), dtype=int) # direction 1=left 2=diag 3=up
    s[0,:] = [i*gap for i in range(0,len(s2)+1)] 
    s[:,0] = [i*gap for i in range(0,len(s1)+1)] 
    d[0,1:],d[1:,0] = 1,3
    for i in range(1,len(s1)+1):
        for j in range(1,len(s2)+1):
            b = s[i-1,j-1] + (match if s1[i-1] == s2[j-1] else mismatch)
            if s[i-1,j] > s[i,j-1]:
                a = s[i-1,j] + gap
                if a > b:
                    s[i,j],d[i,j] = a,3
                    continue
            else:
                a = s[i,j-1] + gap
                if a > b:
                    s[i,j],d[i,j] = a,1
                    continue
            s[i,j],d[i,j] = b,2
    i,j = len(s1),len(s2)
    a1,a2 = [],[]
    while i+j > 0:
        a = d[i,j]
        if a > 1: i-=1; a1.append(s1[i])
        else: a1.append('-')
        if a < 3: j-=1; a2.append(s2[j])
        else: a2.append('-')
    return ''.join(a1[::-1]), ''.join(a2[::-1]), s[-1,-1]